# Audio Feature Analysis Pipeline

## Overview
This Jupyter notebook implements audio feature extraction and analysis based on research by Breyeale & Cook (2008) and Liu et al. (1998). It processes audio from video files to extract discriminative features for content analysis.

### Key Audio Features
- **Volume Contour Analysis** (4 Hz component)
- **Frequency Characteristics**:
  - Centroid
  - Bandwidth
  - Energy ratios (0-630 Hz, 1720-4400 Hz)
- **Temporal Features**:
  - Non-silence ratio
  - Volume dynamic range
  - Volume standard deviation
- **Advanced Features**:
  - MFCC (Mel Frequency Cepstral Coefficients)
  - Zero Crossing Rate (ZCR)

### Prerequisites


In [ ]:
import os
import numpy as np
import librosa
import pandas as pd
import subprocess
from tqdm import tqdm
import warnings



### Technical Requirements
- FFmpeg installation required
- Python libraries: librosa, pandas, numpy
- Input: MP4 video files
- Output: CSV file with extracted audio features

### Feature Extraction Process
The pipeline segments audio into fixed-length windows and extracts multiple acoustic features, providing both instantaneous measurements and statistical aggregations across segments.

In [3]:
def extract_audio_from_video(video_path, audio_path):
    command = ['ffmpeg', '-i', video_path, '-q:a', '0', '-map', 'a', audio_path, '-y']
    subprocess.run(command, check=True)

def calculate_mfcc(y, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs


def analyze_audio_features(segment, sr):
    # Calculate MFCCs and take the mean across time frames
    mfccs = calculate_mfcc(segment, sr)
    mean_mfccs = np.mean(mfccs, axis=1)
    
    # Calculate the frequency component of the volume contour around 4 Hz
    hop_length = 512
    rms = librosa.feature.rms(y=segment, hop_length=hop_length)[0]
    times = librosa.times_like(rms, sr=sr, hop_length=hop_length)
    rms_fft = np.fft.fft(rms)
    freqs = np.fft.fftfreq(len(rms), d=times[1] - times[0])
    idx_4hz = np.argmin(np.abs(freqs - 4.0))
    vol_contour_4hz = np.abs(rms_fft[idx_4hz])
    
    # Calculate the frequency centroid
    centroid = librosa.feature.spectral_centroid(y=segment, sr=sr)[0]
    freq_centroid = np.mean(centroid)
    
    # Calculate the frequency bandwidth
    bandwidth = librosa.feature.spectral_bandwidth(y=segment, sr=sr)[0]
    freq_bandwidth = np.mean(bandwidth)
    
    # Calculate the energy ratio in the range [0–630) Hz
    stft = np.abs(librosa.stft(y=segment))
    freqs = librosa.fft_frequencies(sr=sr)
    total_energy = np.sum(stft)
    low_energy = np.sum(stft[freqs < 630])
    energy_ratio_0_630 = low_energy / total_energy if total_energy > 0 else 0

    # Calculate the energy ratio in the range [630-1720) Hz
    mid_energy = np.sum(stft[(freqs >= 630) & (freqs < 1720)])
    energy_ratio_630_1720 = mid_energy / total_energy if total_energy > 0 else 0
    
    # Calculate the energy ratio in the range [1720-4400) Hz
    high_energy = np.sum(stft[(freqs >= 1720) & (freqs < 4400)])
    energy_ratio_1720_4400 = high_energy / total_energy if total_energy > 0 else 0
    
    # Calculate non-silence ratio
    amplitude_threshold = 0.02  # Amplitude threshold for silence
    non_silence = np.sum(np.abs(segment) > amplitude_threshold)
    non_silence_ratio = non_silence / len(segment)
    
    # Calculate volume dynamic range
    volume_dynamic_range = np.max(rms) - np.min(rms)
    
    # Calculate zero crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(segment)[0]
    mean_zero_crossing_rate = np.mean(zero_crossing_rate)
    
    # Calculate volume standard deviation
    volume_std_dev = np.std(rms)
    
    return {
        "mean_mfccs": mean_mfccs,
        "vol_contour_4hz": vol_contour_4hz,
        "freq_centroid": freq_centroid,
        "freq_bandwidth": freq_bandwidth,
        "energy_ratio_0_630": energy_ratio_0_630,
        "energy_ratio_630_1720": energy_ratio_630_1720,
        "energy_ratio_1720_4400": energy_ratio_1720_4400,
        "non_silence_ratio": non_silence_ratio,
        "volume_dynamic_range": volume_dynamic_range,
        "mean_zero_crossing_rate": mean_zero_crossing_rate,
        "volume_std_dev": volume_std_dev
    }

def analyze_audio_segments(video_path, segment_length=10):
    # Extract the audio from the video using ffmpeg
    audio_path = "temp_audio.wav"
    extract_audio_from_video(video_path, audio_path)
    
    # Load the audio file
    y, sr = librosa.load(audio_path)
    
    # Determine the number of samples in each segment
    segment_samples = int(segment_length * sr)
    
    # Check if the audio is shorter than one segment
    if len(y) < segment_samples:
        # Handle this case by processing the entire audio as one segment
        segments = [y]
    else:
        # Frame the audio into segments
        segments = librosa.util.frame(y, frame_length=segment_samples, hop_length=segment_samples).T
        
    all_features = []
    for segment in segments:
        features = analyze_audio_features(segment, sr)
        all_features.append(features)
    
    return all_features


def calculate_volatility(values):
    differences = np.diff(values)
    volatility = np.std(differences) if not (len(differences) == 0 or np.all(differences == 0)) else np.nan
    return volatility

def aggregate_features(features_list):
    aggregated_features = {}
    # Initialize lists for MFCCs
    mfccs_list = []
    
    # Collect MFCCs separately
    for features in features_list:
        mfccs_list.append(features["mean_mfccs"])
    
    # Convert list of arrays to 2D array
    mfccs_array = np.array(mfccs_list)
    # Compute mean and std across segments for each MFCC coefficient
    aggregated_features["mean_mfccs"] = np.mean(mfccs_array, axis=0)
    aggregated_features["std_mfccs"] = np.std(mfccs_array, axis=0)
    # Compute volatility for MFCCs
    mfccs_diffs = np.diff(mfccs_array, axis=0)
    aggregated_features["volatility_mfccs"] = np.std(mfccs_diffs, axis=0)
    
    # Other scalar features
    scalar_keys = [key for key in features_list[0].keys() if key != "mean_mfccs"]
    for key in scalar_keys:
        all_values = [features[key] for features in features_list]
        mean_value = np.mean(all_values)
        std_value = np.std(all_values)
        volatility_value = calculate_volatility(all_values)
        aggregated_features[f"mean_{key}"] = mean_value
        aggregated_features[f"std_{key}"] = std_value
        aggregated_features[f"volatility_{key}"] = volatility_value
    
    return aggregated_features


def print_audio_features(features):
    formatted_output = "Audio Features:\n-------------------------\n"
    for key, value in features.items():
        if isinstance(value, np.ndarray):
            formatted_output += f"{key}: {value}\n"
        else:
            formatted_output += f"{key}: {value:.4f}\n"
    print(formatted_output)


# Try to load previously processed data, if it exists
try:
    processed_df = pd.read_csv("audio_features.csv")
    processed_video_ids = set(processed_df["video_id"].unique())
    print(f"Resuming from {len(processed_video_ids)} previously processed videos.")
except FileNotFoundError:
    processed_video_ids = set()
    print("Starting fresh, no previously processed videos found.")

Starting fresh, no previously processed videos found.


In [ ]:
video_folder = '../../YouTube_Downloader/Complete_Downloads'

files = os.listdir(video_folder)
mp4_files = [file for file in files if file.endswith('.mp4')]


with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=RuntimeWarning) # ignore runtime warnings (where the output is and should be NaN)

    # Iterate over all .mp4 files in the directory
    for filename in tqdm(mp4_files, total=len(mp4_files)):
        video_path = os.path.join(video_folder, filename)
        video_id = os.path.splitext(filename)[0]  # Get filename without extension

        #### remember to skip already procced videos!!!
        ### !!!!!
    
        # Analyze audio segments and aggregate features
        features_list = analyze_audio_segments(video_path)
        aggregated_features = aggregate_features(features_list)
    
        # Prepare a flat dictionary to store all features
        features_dict = {'video_id': video_id}
    
        # Iterate over the aggregated features
        for feature_name, feature_value in aggregated_features.items():
            if isinstance(feature_value, (list, np.ndarray)):
                # If the feature is an array, expand it into multiple columns
                for idx, val in enumerate(feature_value):
                    new_feature_name = f"{feature_name}_{idx+1}"
                    features_dict[new_feature_name] = val
            else:
                # If the feature is a scalar, add it directly
                features_dict[feature_name] = feature_value
    
        # Append to the CSV, ensuring headers are written only once
        df = pd.DataFrame([features_dict])
        df.to_csv("audio_features.csv", mode="a", header=not bool(processed_video_ids), index=False)
    
        # Add the processed video ID to the set
        processed_video_ids.add(video_id)

  0%|          | 4/20180 [00:18<24:21:54,  4.35s/it]

In [ ]:
df = pd.read_csv("audio_features.csv")
df

In [ ]:
df.info()

# Test (ignore)

In [73]:
# GPT made test using sine wave, silence, and white noise
#!pip install pytest
!pytest test_audio_features.py -v

============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.5, pluggy-1.5.0 -- /opt/conda/bin/python3.12
cachedir: .pytest_cache
rootdir: /work/SDU_data/Video_Processing/Audio
plugins: anyio-4.8.0
collected 6 items                                                              

test_audio_features.py::test_calculate_mfcc PASSED                       [ 16%]
test_audio_features.py::test_analyze_audio_features_silence PASSED       [ 33%]
test_audio_features.py::test_analyze_audio_features_sine_wave PASSED     [ 50%]
test_audio_features.py::test_analyze_audio_features_white_noise PASSED   [ 66%]
test_audio_features.py::test_calculate_volatility PASSED                 [ 83%]
test_audio_features.py::test_aggregate_features PASSED                   [100%]

============================== 6 passed in 2.42s ===============================


In [3]:
import pandas as pd
df = pd.read_csv("audio_features.csv")
df

,video id,mean_mfccs_1,mean_mfccs_2,mean_mfccs_3,mean_mfccs_4,mean_mfccs_5,mean_mfccs_6,mean_mfccs_7,mean_mfccs_8,mean_mfccs_9,...,volatility_non_silence_ratio,mean_volume_dynamic_range,std_volume_dynamic_range,volatility_volume_dynamic_range,mean_mean_zero_crossing_rate,std_mean_zero_crossing_rate,volatility_mean_zero_crossing_rate,mean_volume_std_dev,std_volume_std_dev,volatility_volume_std_dev
0,0fRDt_7MYHU,-225.52790,70.968380,-14.891353,18.770979,3.982643,4.091287,-4.496998,-1.225080,-1.586203,...,0.000000,0.781202,0.005657,0.000000,0.136206,0.012790,0.000000,0.231420,0.003281,0.000000
1,OTPFTVSPos0,-285.71082,135.104780,8.652041,20.172503,-1.294491,9.746528,0.069731,6.280136,1.356463,...,0.123499,0.037035,0.007902,0.011303,0.082760,0.051496,0.051084,0.007464,0.002108,0.002898
2,IAkY5m00rpY,-207.19240,115.454070,21.107504,35.541718,5.330107,10.925244,3.838988,12.151927,-1.893617,...,0.102134,0.157906,0.039820,0.047117,0.052773,0.010787,0.010159,0.030694,0.007858,0.006274
3,D7xAVbO0u28,-247.43733,83.212290,-2.993243,10.897698,11.698050,-3.125383,-9.473722,-0.693984,-14.869232,...,0.050949,0.222378,0.017119,0.031990,0.167690,0.021184,0.021018,0.060138,0.003631,0.005301
4,LiRqu4ipTU0,-294.25955,94.810250,14.404055,25.113530,1.623076,6.898600,-13.839192,-4.361318,1.782887,...,0.080183,0.140592,0.028585,0.033791,0.124468,0.026463,0.032253,0.032705,0.006342,0.007024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20175,eAiN49GfTU8,-234.43370,117.007286,-29.516190,-8.315688,5.963817,-5.721455,-15.401410,9.297347,-21.663116,...,0.105168,0.258137,0.017448,0.024313,0.120590,0.003360,0.004328,0.056077,0.008715,0.017330
20176,rLCKlr624KA,-322.36444,90.833435,9.584511,29.570383,13.435158,6.933914,-9.976146,-2.919571,-7.851149,...,0.109849,0.153206,0.048898,0.052138,0.106206,0.024940,0.023175,0.028981,0.007549,0.007031
20177,LzNIzbf73fM,-61.57134,86.462685,1.548746,19.255110,5.237626,-5.208599,-3.407704,5.174557,-9.819591,...,0.071085,0.195323,0.032740,0.034633,0.134001,0.030565,0.025843,0.038750,0.012241,0.011412
20178,jvLVIcrjcxk,-204.14972,72.094710,-35.422054,36.920280,-34.032074,7.998058,-18.014112,0.458389,-11.179590,...,0.175608,0.115819,0.018567,0.027837,0.177290,0.038975,0.036368,0.028053,0.004701,0.005116
